<a href="https://colab.research.google.com/github/faisalnawazmir/Econometrics-ML_for_Finance/blob/main/Copy_of_Usmanbhai__Fama_MacBeth_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Real example

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm 
from tabulate import tabulate
import time
np.random.seed(12345)

In [ ]:
#copying data from excel file 25 portfoli
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive
file = r'/gdrive/My Drive/data/data_1.xlsx'
df = pd.read_excel(file)
df.head()

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


,Date,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,p15,p16,p17,p18,p19,p20,p21,p22,p23,p24,p25,Mkt_rf,SMB,HML,RMW,CMA,WML
0,2010M01,-0.022449,-0.003999,-0.022411,-0.023178,-0.043471,-0.031099,-0.022518,-0.025929,-0.025970,-0.030001,-0.029863,-0.040193,-0.040045,-0.046545,-0.033626,-0.017201,-0.026599,-0.031670,-0.035366,-0.028325,-0.032098,-0.034478,-0.014938,-0.017491,-0.019410,0.014472,-0.011144,0.012910,-0.044407,-0.040257,-0.035529
1,2010M02,0.049803,0.029144,0.053706,0.075742,0.081194,0.080830,0.090786,0.067251,0.071854,0.057739,0.029606,0.013183,0.005725,0.003897,0.004606,-0.011418,-0.009978,-0.009001,-0.001426,0.009962,0.041415,0.053815,0.082008,0.087472,0.095732,-0.004926,0.058614,0.129087,0.106115,0.058884,0.175846
2,2010M03,0.006199,0.016222,0.009079,-0.002827,-0.015653,-0.010726,-0.014436,-0.021326,-0.030833,-0.027573,-0.015227,-0.010670,-0.014282,-0.008877,-0.003851,0.004704,-0.000805,-0.010352,-0.027967,-0.030412,-0.038455,-0.059199,-0.089694,-0.090903,-0.083014,0.043055,0.002939,-0.048565,0.020677,0.026204,-0.024963
3,2010M04,0.063183,0.050639,0.057855,0.066709,0.067274,0.068328,0.066229,0.063337,0.054213,0.047033,0.021823,0.035438,0.028108,0.015332,0.016480,0.009356,0.016818,0.025236,0.017749,0.017380,0.016594,0.019906,0.024399,0.009386,-0.007756,0.014785,0.042049,0.067846,0.083133,0.092571,0.102913
4,2010M05,-0.254945,-0.231527,-0.221009,-0.222385,-0.221711,-0.207466,-0.203421,-0.194838,-0.195471,-0.184219,-0.176137,-0.165036,-0.156929,-0.152128,-0.145155,-0.130563,-0.128486,-0.122554,-0.121980,-0.117182,-0.118878,-0.118631,-0.088899,-0.082484,-0.043054,-0.121105,-0.018335,-0.004267,-0.073664,-0.006612,-0.070036


In [44]:
#N = 25  # of firms in an economy, here # of portfolios, here len(data)
T = len(df)  # length of periods # here 1136#df.columns.array
#  data
data = df.loc[:,'p1':'p25']
#data.shape#(T,N)
#x,y=data.shape
#N=y#  # of firms in an economy, here # of portfolios, here len(data)
#T = data.shape[0]
N = data.shape[1]

In [ ]:
# simulate factor loadings (ie, coefficients)
b_mkt = np.zeros((1,N)) # true loadings for market factor
b_smb = np.zeros((1,N)) # true loadings for size factor
b_hml = np.zeros((1,N)) # true loadings for value factor

In [ ]:
#  factor returns
mkt = df['Mkt_rf'] # market factor
smb = df['SMB'] # size factor
hml = df['HML'] # value factor

In [ ]:
#data.iloc[:,1]
fctrs = df[["Mkt_rf","SMB","HML"]]
fctrs.head()


,Mkt_rf,SMB,HML
0,0.014472,-0.011144,0.012910
1,-0.004926,0.058614,0.129087
2,0.043055,0.002939,-0.048565
3,0.014785,0.042049,0.067846
4,-0.121105,-0.018335,-0.004267


In [ ]:
# Step 1: time series regression firm by firm
#fctrs = np.c_[mkt, smb, hml]
b = np.zeros(3)
for i in range(N): # firm-by-firm time series regression
  model = LinearRegression()
  model.fit(fctrs,data.iloc[:,i])
  b = np.c_[b,model.coef_]

b  = np.transpose(b[:,1:])
b.shape
#b should be 0.1, 0.2

(25, 3)

In [ ]:
#b is bhat
#b
#b.shape# (N,#factors)
df_bhat = pd.DataFrame(b)
df_bhat.columns = ['b_01','b_02','b_03']
#print(df_bhat.head())

from google.colab import  drive
df_bhat.to_csv('/gdrive/My Drive/data/bhat.csv')

In [ ]:
# Step 2: cross sectional regression time-by-time
f = np.zeros(3)
for i in range(T): # time-by-time cross sectional regression
  model = LinearRegression()
  model.fit(b,data.iloc[i,:])
  f = np.c_[f,model.coef_]

f = np.transpose(f[:,1:])
df_gamma = pd.DataFrame(f)
df_gamma.columns = ['mkt','smb','hml']
print(df_gamma.describe())
print('Median estimate:', np.median(f,axis=0))
print('Standard errors (without Shanken correction):', np.std(f,axis=0)/T**.5)

              mkt         smb         hml
count  120.000000  120.000000  120.000000
mean    -0.000018    0.008240    0.014970
std      0.063501    0.063251    0.149064
min     -0.159709   -0.198469   -0.704467
25%     -0.052223   -0.025270   -0.055652
50%     -0.001835    0.016276    0.015122
75%      0.043834    0.048658    0.109768
max      0.220636    0.156251    0.377468
Median estimate: [-0.0018346   0.01627576  0.01512229]
Standard errors (without Shanken correction): [0.00577258 0.00574989 0.01355077]


In [63]:
mu=np.median(f,axis=0)
sigma=(np.std(f,axis=0))/T**.5
tstat=mu/sigma
tstat
#tstat<1.96

array([-0.31781258,  2.83061892,  1.1159732 ])

In [ ]:
f.shape # (T,#factors)# f or df I assume is gamma hat
df_gamma.to_csv('/gdrive/My Drive/data/gamma.csv')

In [ ]:
df.head()#f and df are same, gamma hat

,mkt,smb,hml
0,0.001553,0.002397,-0.007868
1,0.006439,-0.058778,0.063283
2,0.053603,-0.009954,-0.055991
3,0.047964,-0.075493,-0.108861
4,-0.122230,0.066332,0.105421


In [ ]:
# regrssion using hac newey west method
average(data[:,])

In [59]:
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np
import statsmodels.stats.api as sms

In [ ]:
formula = 'ermsoft ~ ersandp + dprod + dcredit + dinflation + dmoney + dspread + rterm'
results = smf.ols(formula, data).fit(cov_type='HAC',
                                     cov_kwds={'maxlags':6,'use_correction':True})
print(results.summary())

In [50]:
data_a=data.to_numpy()
fctrs_a=fctrs.to_numpy()

In [52]:
#data_a
#fctrs_a

In [54]:
# Compute standard error with bootstrapping (Shanken correction unnecessary)
# One simple bootstrapping 

indices = np.random.randint(low=0,high=T, size=T)
new_data = data_a[indices,:]
new_fctrs = fctrs_a[indices,:]
xx = indices[0]
print(xx)
print(new_data[0,:])
print(data_a[xx,:])


82
[0.16823049 0.16238599 0.15094023 0.1596925  0.14817384 0.14932732
 0.16451328 0.1592505  0.1671695  0.16024045 0.16587053 0.16861627
 0.17092155 0.16034127 0.12956645 0.12172151 0.111725   0.12366251
 0.11783683 0.13533994 0.09188785 0.08800235 0.09615017 0.09690842
 0.09263741]
[0.16823049 0.16238599 0.15094023 0.1596925  0.14817384 0.14932732
 0.16451328 0.1592505  0.1671695  0.16024045 0.16587053 0.16861627
 0.17092155 0.16034127 0.12956645 0.12172151 0.111725   0.12366251
 0.11783683 0.13533994 0.09188785 0.08800235 0.09615017 0.09690842
 0.09263741]


In [56]:
def efhats(data,fctrs): # function to compute average estimated factor return
  T = data.shape[0]
  N = data.shape[1]
  # Step 1: Time series regression
  x = np.c_[np.ones(T),fctrs]
  M = np.linalg.inv(x.T @ x) @ x.T
  bhat = (M @ data)[1:,:].T

  # Step 2: cross sectional regression
  # Include other control variables here (eg, firm characteristics)
  x = np.c_[np.ones(N),bhat]
  M = np.linalg.inv(x.T @ x) @ x.T
  fhat = (M @ data.T)[1:,:].T
  df = pd.DataFrame(fhat)
  df.columns = ['mkt','smb','hml']
  E_fhat = np.average(fhat,axis=0)
  return E_fhat 


In [57]:
E_fhat = efhats(data_a,fctrs_a)
simn = 10**3 # of bootstrapping
t = time.time() # bootstrapping start time 
for i in range(simn): 
  indices = np.random.randint(low=0,high=T,size=T) # resample by rows
  new_data = data_a[indices,:] # resampled stock return data
  new_fctrs = fctrs_a[indices,:] # resampled factor return data
  E_fhat = np.c_[E_fhat,efhats(new_data,new_fctrs)]
  # to check bootstrapping status
  if i % 100 == 0: print(i)

print('Simulation duration:', time.time()-t, 'seconds.')
df = pd.DataFrame(E_fhat.T)
df.columns = ['mkt','smb','hml']
print(df.describe())

0
100
200
300
400
500
600
700
800
900
Simulation duration: 1.1516716480255127 seconds.
               mkt          smb          hml
count  1001.000000  1001.000000  1001.000000
mean      0.000215     0.004480     0.007224
std       0.005698     0.006426     0.015226
min      -0.020977    -0.015944    -0.047733
25%      -0.003539     0.000385    -0.001698
50%       0.000417     0.004377     0.007391
75%       0.004007     0.008324     0.016722
max       0.016910     0.037002     0.080394


In [58]:
print('FM regression esimate of expeced factor returns: \n', np.average(E_fhat.T,axis=0))
print('FM regression esimate of standard errors: \n', np.std(E_fhat.T,axis=0))

FM regression esimate of expeced factor returns: 
 [0.00021453 0.00447986 0.00722402]
FM regression esimate of standard errors: 
 [0.00569556 0.0064231  0.01521866]


https://quant.stackexchange.com/questions/35781/how-to-interpret-the-grs-f-test-values
#grs test
https://stackoverflow.com/questions/24074481/fama-macbeth-regression-in-python-pandas-or-statsmodels